# Gaussian process Coursework

### Acknowledgements

This notebook owes a lot to the excellent resources made available by [gpflow](https://github.com/GPflow/GPflow), in addition to the [original assignment document](http://mlg.eng.cam.ac.uk/teaching/4f13/1920/cw/coursework1.pdf) by Carl Rasmussen. Many parts of it are directly copied from one or other of these resources to avoid re-inventing the wheel.

### Helpful Resources

- I would highly recommend both David Mackay's [video lecture](http://videolectures.net/gpip06_mackay_gpb/) on Gaussian processes and the first chunk of Rich's [video lecture](https://www.youtube.com/watch?v=92-98SYOdlY) if you don't feel this GP stuff has clicked yet.
- Carl Rasmussen's [Gaussian Processes for Machine Learning textbook](http://gaussianprocess.org/gpml/) remains the go-to textbook on the basics. Note that it was published in 2006, so necessarily doesn't cover more recent advances.
- The [GPflow documentation](https://gpflow.readthedocs.io/en/master/?badge=master) is presumably something you will need to consult for this assignment. Likely of particular interest is the documentation on [kernels](https://gpflow.readthedocs.io/en/master/kernel_options.html), [vanilla GP regression](https://gpflow.readthedocs.io/en/master/model_options.html#gp-regression), [the vanilla GP regression notebook](https://nbviewer.jupyter.org/github/GPflow/GPflow/blob/develop/doc/source/notebooks/basics/regression.ipynb) (bits of which I consulted / copied when writing this assignment)
- Also helpful is David Duvenaud's [kernel cookbook](https://www.cs.toronto.edu/~duvenaud/cookbook/index.html) if you want to understand some more about the properties of different kernels.

## Setup

Don't worry too much about this section, it should just run.

Configure your environment.

In [ ]:
import gpflow
import numpy as np
import matplotlib

# The lines below are specific to the notebook format
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
plt = matplotlib.pyplot

Load the data for the assignment into variables `x` and `y`.

In [3]:
import scipy.io
import io
import requests


# To import a .mat file from a URL:
r = requests.get('http://mlg.eng.cam.ac.uk/teaching/4f13/1920/cw/cw1a.mat')
with io.BytesIO(r.content) as f:
    data = scipy.io.loadmat(f)
    x, y = data['x'], data['y']

## a)

Train a GP with a `gpflow.kernels.SquaredExponential` covariance function, with initial parameters `lengthscale=0.35`, `variance=1.0`, and likelihood variance `1.0`, by minimizing the negative log marginal likelihood. Show the 95% predictive error bars and draw some samples from the posterior. Comment on the predictive error bars and the optimized hyperparameters.

(Hint: the GP regression notebook mentioned above will be helpful here, particularly when plotting the results)

What value / extra information does visualising posterior samples provide in addition to the posterior marginals? 
(Hint: try repeating the exercise with `gpflow.kernels.Exponential`)

## b)

Show that by initializing the hyperparameters differently, you can find a different local optimum for
the hyperparameters. Try a range of values. Show the fit. Explain what the model is doing. Which
fit is best, and why? How confident are you about this and why?

## c)

Train instead a GP with a periodic covariance function. Show the fit. Comment on the behaviour
of the error-bars, compared to your fit from a). Do you think the data generating mechanism (apart
from the noise) was really strictly periodic? How confident are you about this, and why? Explain
your reasoning.

## d)

Generate random (essentially) noise free functions evaluated at x = linspace(-5,5,200)’; from a GP with the following covariance function: {@covProd, {@covPeriodic, @covSEiso}}, with covari- ance hyperparameters hyp.cov = [-0.5 0 0 2 0]. In order to apply the Cholesky decomposition to the covariance matrix, you may have to add a small diagonal matrix, for example 1e-6*eye(200), why? Plot some sample functions. Explain the relationship between the properties of those random functions and the form of the covariance function.

## e)

## f)

Download additional data set

In [7]:
r = requests.get('http://mlg.eng.cam.ac.uk/teaching/4f13/1920/cw/cw1e.mat')
with io.BytesIO(r.content) as f:
    data = scipy.io.loadmat(f)
    x_ml, y_ml = data['x'], data['y']